In [3]:
%run "Clusterly.ipynb" # last year

In [4]:
# Import libraries 
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from pandas.tseries.offsets import MonthBegin, MonthEnd
import utilities as nori
from utilities import *
import dask.dataframe as dd
import sqlalchemy as db

# Connect to ODBC
import pyodbc
conn = pyodbc.connect(DSN='Radar',UID='ruser',PWD='8uzUTHub')

currentDt = pd.datetime.now().date()
date1 = (currentDt + MonthBegin(-2)).strftime('%m/%d/%Y 00:0:0')
date2 = (currentDt + MonthEnd(6)).strftime('%m/%d/%Y 00:0:0')

transDt1 = (currentDt - pd.Timedelta(pd.offsets.Day(2))).strftime('%m/%d/%Y 00:0:0')
transDt2 = (currentDt - pd.Timedelta(pd.offsets.Day(9))).strftime('%m/%d/%Y 00:0:0')
transDt3 = (currentDt - pd.Timedelta(pd.offsets.Day(16))).strftime('%m/%d/%Y 00:0:0')

# Published Fare
query1 = """SELECT
  REVENUE.RecordLocator AS PNR,
  TRVL_CALENDAR.DateDate AS TrvlDt,
  BKG_CALENDAR.DateDate AS BkDt,
  AIRPORT_POO.AirportCode AS Org,
  AIRPORT_POD.AirportCode AS Dest,
  AIRPORT_POO.IsDomestic AS OrgDom,
  AIRPORT_POD.IsDomestic AS DestDom,
  AIRPORT_LEG_DEP.AirportCode AS Dept,
  AIRPORT_LEG_ARL.AirportCode AS Arr,
  (ROUTE_LEG.IndustryMiles) AS Miles,
  count( distinct REVENUE.PassengerSrcID ) AS Pax,
  Sum(REVENUE.RevenueAmount) AS Rev
FROM
  CalendarDim  BKG_CALENDAR INNER JOIN RevenueFact  REVENUE ON (REVENUE.BookingDateSID=BKG_CALENDAR.DateSID)
   INNER JOIN ClassOfServiceDim  CLASSOFSERVICE ON (REVENUE.ClassOfServiceSID=CLASSOFSERVICE.ClassOfServiceSID  AND  CLASSOFSERVICE.IsCurrent=1)
   INNER JOIN TransactionStatusDim  TRANS_STATUS ON (REVENUE.TransactionStatusSID=TRANS_STATUS.TransactionStatusSID)
   INNER JOIN TransactionTypeDim  TRANS_TYPE ON (TRANS_TYPE.TransactionTypeSID=REVENUE.TransactionTypeSID)
   INNER JOIN CalendarDim  TRVL_CALENDAR ON (REVENUE.DepartureDateSID=TRVL_CALENDAR.DateSID)
   INNER JOIN RouteDim  ROUTE_LEG ON (REVENUE.LegRouteSID=ROUTE_LEG.RouteSID  AND  ROUTE_LEG.IsCurrent=1)
   INNER JOIN AirportDim  AIRPORT_LEG_DEP ON (ROUTE_LEG.RouteDeprAirportSID=AIRPORT_LEG_DEP.AirportSID)
   INNER JOIN AirportDim  AIRPORT_LEG_ARL ON (ROUTE_LEG.RouteArrvAirportSID=AIRPORT_LEG_ARL.AirportSID)
   INNER JOIN AirportDim  AIRPORT_POO ON (AIRPORT_POO.AirportSID=REVENUE.POO_SID)
   INNER JOIN AirportDim  AIRPORT_POD ON (AIRPORT_POD.AirportSID=REVENUE.POD_SID)
   INNER JOIN ChannelDim  BKG_CHANNEL ON (BKG_CHANNEL.ChannelSID=REVENUE.BookingChannelSID)
  
WHERE
  (
   TRVL_CALENDAR.DateDate  BETWEEN  '%s'  AND  '%s'
   AND
   BKG_CALENDAR.DateDate <= '%s'
   AND
   AIRPORT_POO.AirportCode  <>  'UNK'
   AND
   AIRPORT_POD.AirportCode  <>  'UNK'
   AND
   TRANS_STATUS.StatusTier3  IN  ( 'Balance Due','Boarded (flown)','Booked','Expired','No Show','Unknown'  )
   AND
   TRANS_TYPE.TypeTier3  =  'Published Fare'
   AND
   CLASSOFSERVICE.bool_NonRev  =  0
  )
GROUP BY
  REVENUE.RecordLocator, 
  TRVL_CALENDAR.DateDate, 
  BKG_CALENDAR.DateDate, 
  AIRPORT_POO.AirportCode, 
  AIRPORT_POD.AirportCode, 
  AIRPORT_POO.IsDomestic, 
  AIRPORT_POD.IsDomestic, 
  AIRPORT_LEG_DEP.AirportCode, 
  AIRPORT_LEG_ARL.AirportCode, 
  (ROUTE_LEG.IndustryMiles)"""%(date1,date2,transDt1)
df1 = pd.read_sql(query1,conn,parse_dates=['TrvlDt','BkDt'])

df1 = nori.connect(df1) # Connect
df1 = df1.groupby(['PNR','TrvlDt','BkDt','Org','Dest','OrgDom','DestDom'],as_index=False).agg({'Pax':'max','Rev':'sum','Connect':'max','Miles':'sum'})

# Baggage
query2 = """SELECT
  REVENUE.RecordLocator AS PNR,
  TRVL_CALENDAR.DateDate AS TrvlDt,
  TRANS_CALENDAR.DateDate AS TransDt,
  AIRPORT_POO.AirportCode AS Org,
  AIRPORT_POD.AirportCode AS Dest,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG1' THEN REVENUE.PassengerSrcID ELSE NULL END) as Bag1,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG1' THEN REVENUE.RevenueAmount ELSE 0 END) as Bag1Rev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG2' THEN REVENUE.PassengerSrcID ELSE NULL END) as Bag2,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG2' THEN REVENUE.RevenueAmount ELSE 0 END) as Bag2Rev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 IN ( 'Gate Carry-On','Carry-On' ) THEN REVENUE.PassengerSrcID ELSE NULL END) as CarryOn,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 IN ( 'Gate Carry-On','Carry-On' ) THEN REVENUE.RevenueAmount ELSE 0 END) as CarryOnRev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 IN ('Gate Carry-On','Carry-On','BAG1','BAG2','BAG3','BAG4','BAG5') THEN REVENUE.PassengerSrcID ELSE NULL END) as Cum,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 IN ('Gate Carry-On','Carry-On','BAG1','BAG2','BAG3','BAG4','BAG5') THEN REVENUE.RevenueAmount ELSE 0 END) as CumRev

FROM
  ChannelDim  TRANS_CHANNEL INNER JOIN RevenueFact  REVENUE ON (REVENUE.TransactionChannelSID=TRANS_CHANNEL.ChannelSID)
   INNER JOIN ClassOfServiceDim  CLASSOFSERVICE ON (REVENUE.ClassOfServiceSID=CLASSOFSERVICE.ClassOfServiceSID  AND  CLASSOFSERVICE.IsCurrent=1)
   INNER JOIN TransactionStatusDim  TRANS_STATUS ON (REVENUE.TransactionStatusSID=TRANS_STATUS.TransactionStatusSID)
   INNER JOIN TransactionTypeDim  TRANS_TYPE ON (TRANS_TYPE.TransactionTypeSID=REVENUE.TransactionTypeSID)
   INNER JOIN CalendarDim  TRVL_CALENDAR ON (REVENUE.DepartureDateSID=TRVL_CALENDAR.DateSID)
   INNER JOIN CalendarDim  TRANS_CALENDAR ON (REVENUE.TransactionDateSID=TRANS_CALENDAR.DateSID)
   INNER JOIN RouteDim  ROUTE_LEG ON (REVENUE.LegRouteSID=ROUTE_LEG.RouteSID  AND  ROUTE_LEG.IsCurrent=1)
   INNER JOIN AirportDim  AIRPORT_LEG_DEP ON (ROUTE_LEG.RouteDeprAirportSID=AIRPORT_LEG_DEP.AirportSID)
   INNER JOIN AirportDim  AIRPORT_LEG_ARL ON (ROUTE_LEG.RouteArrvAirportSID=AIRPORT_LEG_ARL.AirportSID)
   INNER JOIN AirportDim  AIRPORT_POO ON (AIRPORT_POO.AirportSID=REVENUE.POO_SID)
   INNER JOIN AirportDim  AIRPORT_POD ON (AIRPORT_POD.AirportSID=REVENUE.POD_SID)
  
WHERE
  (
   TRVL_CALENDAR.DateDate  BETWEEN  '%s'  AND  '%s'
   AND
   TRANS_CALENDAR.DateDate <= '%s'
   AND
   AIRPORT_POO.AirportCode  <>  'UNK'
   AND
   AIRPORT_POD.AirportCode  <>  'UNK'
   AND
   TRANS_STATUS.StatusTier3  IN  ( 'Balance Due','Boarded (flown)','Booked','Expired','No Show','Unknown'  )
   AND
   TRANS_TYPE.TypeTier3  IN  ( 'BAG1','BAG2','BAG3','BAG4','BAG5','Carry-On','Gate Carry-On'  )
   AND
   rtrim(REVENUE.FeeCode)  NOT IN  ( 'BMLC','BML1','BML2','BAGE','BAGN','BAGP'  )
   AND
   CLASSOFSERVICE.bool_NonRev  =  0
  )
GROUP BY
  REVENUE.RecordLocator, 
  TRVL_CALENDAR.DateDate,
  TRANS_CALENDAR.DateDate,
  AIRPORT_POO.AirportCode, 
  AIRPORT_POD.AirportCode
HAVING
  Sum(REVENUE.RevenueAmount)  >=  0"""%(date1,date2,transDt1)
df2 = pd.read_sql(query2,conn)

ddf1 = df1[(df1.BkDt <= transDt1)] # Conditional selection by booking and transaction date
ddf2 = df1[(df1.BkDt <= transDt2)]
ddf3 = df1[(df1.BkDt <= transDt3)]

ddf4 = df2[(df2.TransDt <= transDt1)] 
ddf5 = df2[(df2.TransDt <= transDt2)]
ddf6 = df2[(df2.TransDt <= transDt3)]

df1 = pd.merge(ddf1,ddf4,how='left',on=['PNR','TrvlDt','Org','Dest']) # Merge and convert to Dask
df2 = pd.merge(ddf2,ddf5,how='left',on=['PNR','TrvlDt','Org','Dest'])
df3 = pd.merge(ddf3,ddf6,how='left',on=['PNR','TrvlDt','Org','Dest'])

df1,df2,df3 = [dd.from_pandas(df,npartitions=3) for df in [df1,df2,df3]]

# Data Wrangling
df1,df2,df3 = [df.pipe(month) for df in [df1,df2,df3]] # Month Extract
df1,df2,df3 = [df.pipe(diff) for df in [df1,df2,df3]] # DTD
df1,df2,df3 = [df.pipe(journey) for df in [df1,df2,df3]] # Domestic Indicator 

df1,df2,df3 = [df.assign(Domestic = df.Domestic.apply(domestic,meta='bool')) for df in [df1,df2,df3]]

trip1,trip2,trip3 = [df.pipe(duration) for df in [df1,df2,df3]] # Trip Duration
df1 = df1.merge(trip1,how='left',on='PNR')
df2 = df2.merge(trip2,how='left',on='PNR')
df3 = df3.merge(trip3,how='left',on='PNR')

df1,df2,df3 = [df.assign(Miles_Group = df.Miles.apply(miles,meta='object')) for df in [df1,df2,df3]] # Feature binning
df1,df2,df3 = [df.assign(DTD_Group = df.DTD.apply(dtd,meta='object')) for df in [df1,df2,df3]]
df1,df2,df3 = [df.assign(Pax_Group = df.Pax.apply(pax,meta='object')) for df in [df1,df2,df3]]

df1,df2,df3 = [df.pipe(cluster) for df in [df1,df2,df3]] # Cluster
df1,df2,df3 = [df.fillna(0) for df in [df1,df2,df3]] # Fill missing values

df1Total, df2Total, df3Total = [
    df.groupby(
        ['TrvlMo']
    ).agg({'Connect':'count'}).reset_index().rename(
        columns={'Connect':'ConnectTotal'}) for df in [df1,df2,df3]]

df1 = nori.group([df1,df1Total]) # Merge
df2 = nori.group([df2,df2Total])
df3 = nori.group([df3,df3Total])

# Groupby 
df1, df2, df3 = [
    df.groupby(
        ['Cluster','Miles_Group','DTD_Group','Pax_Group','TrvlMo','Domestic']
    ).agg({'PNR':'count','Pax':'sum','Rev':'mean','Duration':'mean', 
           'Connect':'sum','ConnectTotal':'count','CarryOn':'sum','CarryOnRev':'sum','Bag1':'sum',
           'Bag1Rev':'sum','Bag2':'sum','Bag2Rev':'sum','Cum':'sum','CumRev':'sum'}).reset_index() for df in [df1,df2,df3]]

df1Total, df2Total, df3Total = [
    df.groupby(
        ['TrvlMo']
    ).agg({'PNR':'sum'}).reset_index().rename(
        columns={'PNR':'PNRTotal'}) for df in [df1,df2,df3]]

df1 = nori.group([df1,df1Total]) # Merge
df2 = nori.group([df2,df2Total])
df3 = nori.group([df3,df3Total])

df1,df2,df3 = [df.pipe(ratios) for df in [df1,df2,df3]] 
df1,df2,df3 = [df.pipe(bag1) for df in [df1,df2,df3]]
df1,df2,df3 = [df.pipe(bag2) for df in [df1,df2,df3]]
df1,df2,df3 = [df.pipe(carryon) for df in [df1,df2,df3]]
df1,df2,df3 = [df.pipe(cum) for df in [df1,df2,df3]]

# df1,df2,df3 = map(functioncaller, [ratios,bag1,bag2,carryon,checked,cum])

df4 = dd.read_parquet('ly {}.parquet'.format(currentDt.strftime('%Y-%m-%d'))) # Load parquet files
df5 = dd.read_parquet('lyw {}.parquet'.format(currentDt.strftime('%Y-%m-%d')))
df6 = dd.read_parquet('lytw {}.parquet'.format(currentDt.strftime('%Y-%m-%d')))
df7 = dd.read_parquet('lyf {}.parquet'.format((currentDt + MonthBegin(-1)).strftime('%Y-%m-%d')))

# Rename columns
df2 = df2.rename(columns={'PNR':'PNRLW','Pax':'PaxLW','Rev':'RevLW','Duration':'DurationLW','ConnectTotal':'ConnectTotalLW','Connect':'ConnectLW',
           'CarryOn':'CarryOnLW','CarryOnRev':'CarryOnRevLW','Bag1':'Bag1LW','Bag1Rev':'Bag1RevLW','Bag2':'Bag2LW','Bag2Rev':'Bag2RevLW',
           'Cum':'CumLW','CumRev':'CumRevLW','PNRTotal':'PNRTotalLW','%Connect':'%ConnectLW','CarryOn/Bag1':'CarryOn/Bag1LW',
           'Bag2/Bag1':'Bag2/Bag1LW','%PNR':'%PNRLW','Bag1TR':'Bag1TRLW','Bag1RPP':'Bag1RPPLW','Bag1BR':'Bag1BRLW','Bag2TR':'Bag2TRLW',
           'Bag2RPP':'Bag2RPPLW','Bag2BR':'Bag2BRLW','CarryOnTR':'CarryOnTRLW','CarryOnRPP':'CarryOnRPPLW','CarryOnBR':'CarryOnBRLW',
           'CumTR':'CumTRLW','CumRPP':'CumRPPLW','CumBR':'CumBRLW'})

df3 = df3.rename(columns={'PNR':'PNRLTW','Pax':'PaxLTW','Rev':'RevLTW','Duration':'DurationLTW','ConnectTotal':'ConnectTotalLTW','Connect':'ConnectLTW',
           'CarryOn':'CarryOnLTW','CarryOnRev':'CarryOnRevLTW','Bag1':'Bag1LTW','Bag1Rev':'Bag1RevLTW','Bag2':'Bag2LTW','Bag2Rev':'Bag2RevLTW',
           'Cum':'CumLTW','CumRev':'CumRevLTW','PNRTotal':'PNRTotalLTW','%Connect':'%ConnectLTW','CarryOn/Bag1':'CarryOn/Bag1LTW',
           'Bag2/Bag1':'Bag2/Bag1LTW','%PNR':'%PNRLTW','Bag1TR':'Bag1TRLTW','Bag1RPP':'Bag1RPPLTW','Bag1BR':'Bag1BRLTW','Bag2TR':'Bag2TRLTW',
           'Bag2RPP':'Bag2RPPLTW','Bag2BR':'Bag2BRLTW','CarryOnTR':'CarryOnTRLTW','CarryOnRPP':'CarryOnRPPLTW','CarryOnBR':'CarryOnBRLTW',
           'CumTR':'CumTRLTW','CumRPP':'CumRPPLTW','CumBR':'CumBRLTW'})

df4 = df4.rename(columns={'PNR':'PNRLY','Pax':'PaxLY','Rev':'RevLY','Duration':'DurationLY','ConnectTotal':'ConnectTotalLY','Connect':'ConnectLY',
           'CarryOn':'CarryOnLY','CarryOnRev':'CarryOnRevLY','Bag1':'Bag1LY','Bag1Rev':'Bag1RevLY','Bag2':'Bag2LY','Bag2Rev':'Bag2RevLY',
           'Cum':'CumLY','CumRev':'CumRevLY','PNRTotal':'PNRTotalLY','%Connect':'%ConnectLY','CarryOn/Bag1':'CarryOn/Bag1LY',
           'Bag2/Bag1':'Bag2/Bag1LY','%PNR':'%PNRLY','Bag1TR':'Bag1TRLY','Bag1RPP':'Bag1RPPLY','Bag1BR':'Bag1BRLY','Bag2TR':'Bag2TRLY',
           'Bag2RPP':'Bag2RPPLY','Bag2BR':'Bag2BRLY','CarryOnTR':'CarryOnTRLY','CarryOnRPP':'CarryOnRPPLY','CarryOnBR':'CarryOnBRLY',
           'CumTR':'CumTRLY','CumRPP':'CumRPPLY','CumBR':'CumBRLY'})

df5 = df5.rename(columns={'PNR':'PNRLYW','Pax':'PaxLYW','Rev':'RevLYW','Duration':'DurationLYW','ConnectTotal':'ConnectTotalLYW','Connect':'ConnectLYW',
           'CarryOn':'CarryOnLYW','CarryOnRev':'CarryOnRevLYW','Bag1':'Bag1LYW','Bag1Rev':'Bag1RevLYW','Bag2':'Bag2LYW','Bag2Rev':'Bag2RevLYW',
           'Cum':'CumLYW','CumRev':'CumRevLYW','PNRTotal':'PNRTotalLYW','%Connect':'%ConnectLYW','CarryOn/Bag1':'CarryOn/Bag1LYW',
           'Bag2/Bag1':'Bag2/Bag1LYW','%PNR':'%PNRLYW','Bag1TR':'Bag1TRLYW','Bag1RPP':'Bag1RPPLYW','Bag1BR':'Bag1BRLYW','Bag2TR':'Bag2TRLYW',
           'Bag2RPP':'Bag2RPPLYW','Bag2BR':'Bag2BRLYW','CarryOnTR':'CarryOnTRLYW','CarryOnRPP':'CarryOnRPPLYW','CarryOnBR':'CarryOnBRLYW',
           'CumTR':'CumTRLYW','CumRPP':'CumRPPLYW','CumBR':'CumBRLYW'})

df6 = df6.rename(columns={'PNR':'PNRLYTW','Pax':'PaxLYTW','Rev':'RevLYTW','Duration':'DurationLYTW','ConnectTotal':'ConnectTotalLYTW','Connect':'ConnectLYTW',
           'CarryOn':'CarryOnLYTW','CarryOnRev':'CarryOnRevLYTW','Bag1':'Bag1LYTW','Bag1Rev':'Bag1RevLYTW','Bag2':'Bag2LYTW','Bag2Rev':'Bag2RevLYTW',
           'Cum':'CumLYTW','CumRev':'CumRevLYTW','PNRTotal':'PNRTotalLYTW','%Connect':'%ConnectLYTW','CarryOn/Bag1':'CarryOn/Bag1LYTW',
           'Bag2/Bag1':'Bag2/Bag1LYTW','%PNR':'%PNRLYTW','Bag1TR':'Bag1TRLYTW','Bag1RPP':'Bag1RPPLYTW','Bag1BR':'Bag1BRLYTW','Bag2TR':'Bag2TRLYTW',
           'Bag2RPP':'Bag2RPPLYTW','Bag2BR':'Bag2BRLYTW','CarryOnTR':'CarryOnTRLYTW','CarryOnRPP':'CarryOnRPPLYTW','CarryOnBR':'CarryOnBRLYTW',
           'CumTR':'CumTRLYTW','CumRPP':'CumRPPLYTW','CumBR':'CumBRLYTW'})

df7 = df7.rename(columns={'PNR':'PNRLYF','Pax':'PaxLYF','Rev':'RevLYF','Duration':'DurationLYF','CarryOn':'CarryOnLYF','CarryOnRev':'CarryOnRevLYF','Bag1':'Bag1LYF','Bag1Rev':'Bag1RevLYF','Bag2':'Bag2LYF','Bag2Rev':'Bag2RevLYF',
           'Cum':'CumLYF','CumRev':'CumRevLYF','Bag1TR':'Bag1TRLYF','Bag1RPP':'Bag1RPPLYF','Bag1BR':'Bag1BRLYF','Bag2TR':'Bag2TRLYF',
           'Bag2RPP':'Bag2RPPLYF','Bag2BR':'Bag2BRLYF','CarryOnTR':'CarryOnTRLYF','CarryOnRPP':'CarryOnRPPLYF','CarryOnBR':'CarryOnBRLYF',
           'CumTR':'CumTRLYF','CumRPP':'CumRPPLYF','CumBR':'CumBRLYF'})

"""replacement_dict = {'H1': 'H0', 'H2': 'H20', 'H3': 'H7'} # Automate renaming
for k,v in multi.items():
  multi[k] = v.rename(columns = replacement_dict)"""

#print(list(df1.columns & df2.columns & df3.columns & df4.columns & df5.columns & df6.columns & df7.columns)) # Columns in common

import functools # Merge multiple dataframes
dataframes = [df1,df2,df3,df4,df5,df6,df7]
ddf = functools.reduce(lambda  left,right: dd.merge(left,right,on=['Cluster','Miles_Group','DTD_Group','Pax_Group','Domestic','TrvlMo'],
                                                                    how='outer'), dataframes)
ddf = nori.yoy(ddf) # Create metrics
ddf = nori.wow(ddf) 
ddf = nori.wo2w(ddf) 

# Surcharge DB contains cluster table with Cluster, Surcharge, DOI
engine = db.create_engine('sqlite:///surcharge.sqlite') # Query surcharges per cluster
conn = engine.connect()
meta = db.MetaData()
cluster = db.Table('cluster',meta,autoload=True,autoload_with=engine)

query = db.select([cluster])
ResultProxy = conn.execute(query)
ResultSet = ResultProxy.fetchall()

df = pd.DataFrame(ResultSet) # Convert to DataFrame
df.columns = ResultSet[0].keys()

ddf = ddf.merge(df,how='left',on=['Cluster']) # Dynamic surcharges and static gap as of 2/7/19

dtd = [(0,21),(22,45),(46,60),(61,120)] # Priceable
def price(dtd): 
    for i, e in enumerate(dtd): 
        dtd[i] = set(((currentDt + timedelta(days=e[0])).month,(currentDt + timedelta(days=e[1])).month))
        i+=1
    return dtd
price(dtd)

priceDict = {
'0-21':dtd[0],
'22-45':dtd[1],
'46-60':dtd[2],
'60+':dtd[3]
 }

# Dask Delayed
ddf = ddf.compute() # Revert to pandas DF

ddf['Priceable'] = np.where((ddf['DTD_Group'] == '0-21') & (ddf['TrvlMo'].isin(dtd[0])),True,
                       np.where((ddf['DTD_Group'] == '22-45') & (ddf['TrvlMo'].isin(dtd[1])),True,
                           np.where((ddf['DTD_Group'] == '46-60') & (ddf['TrvlMo'].isin(dtd[2])),True,
                               np.where((ddf['DTD_Group'] == '60+') & (ddf['TrvlMo'].isin(dtd[3])),True,False))))

# Demand DoD Trend
# Earliest cluster pax, revenue, bag, bag revenue and DoD build for last year


# Report Formatting
ddf = ddf[['Cluster','Domestic','TrvlMo', '%PNR', '%PNRYoY','Rev','RevLY','FareYoY','Duration','DurationLY','DurationYoY','Miles_Group','DTD_Group','Pax_Group',
           'Priceable','Surcharge','DOI','%Connect','%ConnectLY','%ConnectYoY','Pax','PaxLY','CarryOn/Bag1','CarryOn/Bag1LY',
           'Bag2/Bag1','Bag2/Bag1LY','Cum','CumLY','CumRev','CumRevLY','CumRPP','CumRPPLY','CumRPPLYF','CumRPPYoY','CumRPPWoW',
           'CumRPPWo2W','CumTR','CumTRLY','CumTRLYF','CumTRYoY','CumTRWoW','CumTRWo2W','CumBR','CumBRLY','CumBRLYF','CumBRYoY',
           'CumBRWoW','CumBRWo2W','CarryOn','CarryOnLY','CarryOnRev','CarryOnRevLY','CarryOnRPP','CarryOnRPPLY','CarryOnRPPLYF','CarryOnRPPYoY',
           'CarryOnRPPWoW','CarryOnRPPWo2W','CarryOnTR','CarryOnTRLY','CarryOnTRLYF','CarryOnTRYoY','CarryOnTRWoW','CarryOnTRWo2W',
           'CarryOnBR','CarryOnBRLY','CarryOnBRLYF','CarryOnBRYoY','CarryOnBRWoW','CarryOnBRWo2W','Bag1','Bag1LY','Bag1Rev','Bag1RevLY',
           'Bag1RPP','Bag1RPPLY','Bag1RPPLYF','Bag1RPPYoY','Bag1RPPWoW','Bag1RPPWo2W','Bag1TR','Bag1TRLY','Bag1TRLYF','Bag1TRYoY','Bag1TRWoW',
           'Bag1TRWo2W','Bag1BR','Bag1BRLY','Bag1BRLYF','Bag1BRYoY','Bag1BRWoW','Bag1BRWo2W','Bag2','Bag2LY','Bag2Rev','Bag2RevLY','Bag2RPP',
           'Bag2RPPLY','Bag2RPPLYF','Bag2RPPYoY','Bag2RPPWoW','Bag2RPPWo2W','Bag2TR','Bag2TRLY','Bag2TRLYF','Bag2TRYoY','Bag2TRWoW','Bag2TRWo2W',
           'Bag2BR','Bag2BRLY','Bag2BRLYF','Bag2BRYoY','Bag2BRWoW','Bag2BRWo2W']]

ddf.columns = ['Cluster','Domestic','Travel Month', '%PNR', '%PNR YoY', 'Average Fare','Average Fare LY','Average Fare YoY','Trip Duration','Trip Duration LY',
               'Trip Duration YoY','Miles','DTD','Pax per PNR','Priceable','Surcharge','DOI','%Connect','%Connect LY',
               '%Connect YoY','Total Pax','Total Pax LY','CarryOn/Bag1','CarryOn/Bag1 LY','Bag2/Bag1','Bag2/Bag1 LY',
               'Cumulative','Cumulative LY','Cumulative Rev','Cumulative Rev LY','Cumulative RPP','Cumulative RPP LY','Cumulative RPP LYF',
               'Cumulative RPP YoY','Cumulative RPP WoW','Cumulative RPP Wo2W','Cumulative TR','Cumulative TR LY','Cumulative TR LYF',
               'Cumulative TR YoY','Cumulative TR WoW','Cumulative TR Wo2W','Cumulative BR','Cumulative BR LY','Cumulative BR LYF',
               'Cumulative BR YoY','Cumulative BR WoW','Cumulative BR Wo2W','CarryOn','CarryOn LY','CarryOn Rev','CarryOn Rev LY',
               'CarryOn RPP','CarryOn RPP LY','CarryOn RPP LYF','CarryOn RPP YoY','CarryOn RPP WoW','CarryOn RPP Wo2W','CarryOn TR',
               'CarryOn TR LY','CarryOn TR LYF','CarryOn TR YoY','CarryOn TR WoW','CarryOn TR Wo2W','CarryOn BR','CarryOn BR LY',
               'CarryOn BR LYF','CarryOn BR YoY','CarryOn BR WoW','CarryOn BR Wo2W','Bag1','Bag1 LY','Bag1 Rev','Bag1 Rev LY','Bag1 RPP',
               'Bag1 RPP LY','Bag1 RPP LYF','Bag1 RPP YoY','Bag1 RPP WoW','Bag1 RPP Wo2W','Bag1 TR','Bag1 TR LY','Bag1 TR LYF','Bag1 TR YoY',
               'Bag1 TR WoW','Bag1 TR Wo2W','Bag1 BR','Bag1 BR LY','Bag1 BR LYF','Bag1 BR YoY','Bag1 BR WoW','Bag1 BR Wo2W','Bag2','Bag2 LY',
               'Bag2 Rev','Bag2 Rev LY','Bag2 RPP','Bag2 RPP LY','Bag2 RPP LYF','Bag2 RPP YoY','Bag2 RPP WoW','Bag2 RPP Wo2W',
               'Bag2 TR','Bag2 TR LY','Bag2 TR LYF','Bag2 TR YoY','Bag2 TR WoW','Bag2 TR Wo2W','Bag2 BR','Bag2 BR LY','Bag2 BR LYF',
               'Bag2 BR YoY','Bag2 BR WoW','Bag2 BR Wo2W']

# Export to csv 
ddf.set_index('Cluster').sort_values(by=['Cluster','Domestic','Travel Month']).to_csv('BagClusterReport {}.csv'.format(currentDt.strftime('%Y-%m-%d')))

In [13]:
ddf.head()

,Cluster,Domestic,Travel Month,%PNR,%PNR YoY,Average Fare,Average Fare LY,Average Fare YoY,Trip Duration,Trip Duration LY,...,Bag2 TR LYF,Bag2 TR YoY,Bag2 TR WoW,Bag2 TR Wo2W,Bag2 BR,Bag2 BR LY,Bag2 BR LYF,Bag2 BR YoY,Bag2 BR WoW,Bag2 BR Wo2W
0,Long_0-21_Couple,False,4,0.004160,0.001813,132.036645,136.341641,-4.304996,5.679120,6.431158,...,0.065997,-0.018024,-0.018254,-0.014070,48.580741,46.419481,46.400086,2.161260,2.554613,2.562674
1,Long_0-21_Couple,True,4,0.011733,-0.003239,172.594359,152.000978,20.593382,2.024495,2.393457,...,0.016069,-0.000974,-0.000130,0.001127,50.298069,49.250420,49.212603,1.047649,0.960885,1.068291
2,Long_0-21_Group,False,4,0.001534,0.000825,241.677602,235.265507,6.412096,5.117737,5.928707,...,0.054280,-0.025737,-0.022357,-0.020993,48.941287,46.937862,46.937862,2.003426,3.385700,2.319573
3,Long_0-21_Group,True,4,0.003790,-0.000794,279.875696,257.392828,22.482869,2.271165,2.437739,...,0.010789,0.001283,0.001076,0.000772,51.134318,48.320090,48.272049,2.814229,2.649764,1.569325
4,Long_0-21_Single,False,4,0.018182,0.007343,72.409670,80.169046,-7.759376,6.191296,7.423948,...,0.099044,-0.020512,-0.017957,-0.011965,50.346291,47.210448,47.146847,3.135843,3.223439,3.576877
